# Hello, Chameleon

In this notebook, you will pick up where you left off after creating a Chameleon account, joining a Chameleon project, and preparing key pair. Now, you will learn how to:

-   Reserve resources in Chameleon
-   Access your reserved resources over SSH
-   Execute commands on your resources
-   Retrieving files saved on Chameleon resources
-   Extend your Chameleon lease (in case you need more time) or delete it (in case you finish early)

## Exercise: reserve resources

Whenever you run an experiment on Chameleon, you will

1.  Open a Python notebook, which includes commands to reserve the resources (VMs, bare metal servers, or networks) that you need for your experiment. Run these commands.
2.  Wait until the resources in your experiment are ready to log in.
3.  Log in to the resources and run your experiment (either by executing commands in the notebook, or by using SSH in a terminal and running commands in those SSH sessions).

Also, when you finish an experiment and have saved all the data somewhere safe, you will *delete* the resources in your experiment to free them for use by other experimenters.

In this exercise, we will reserve a resource on Chameleon.

### Generating a Virtual Machine on chameleon

The below cells will take project_name and exp_name as input from the user to configure the experiment environment.

In [ ]:
import chi,os
project_name = "CHI-XXXXXX"
os.environ["OS_PROJECT_NAME"] = project_name
chi.set("project_name", project_name)  
chi.use_site("KVM@TACC")

In [ ]:
exp_name = ""
user = os.getenv("USER")
server_name = f"{exp_name}_{user}"


### Assigning flavor and image_name

Running this cell will show tha available images that could be used in our vm.

In [ ]:
%%bash
openstack image list

Select the image which you want to use and assign it to the image_name variable in the cell below. Here we have used CC-Ubuntu20.04 but you can use different according to your need.

In [ ]:
flavor = "m1.small"
image_name = "CC-Ubuntu20.04"


### Creating the server

In [ ]:
import chi.server

server = chi.server.create_server(server_name, 
                                  image_name=image_name, 
                                  flavor_name=flavor)

server_id = server.id
chi.server.wait_for_active(server_id)

### Attaching a floating IP

At KVM@TACC, since there are no reservations, we can easily obtain a floating IP address from the available pool without any prior booking. However, it’s crucial to keep in mind that the pool of floating IPs is limited. Therefore, it’s advisable to be mindful of your usage and not allocate more floating IPs than necessary, considering the other researchers who also need to utilize them.

In case you require multiple VMs for your experiment, a practical approach is to connect them all on one network. By doing so, you can use a single floating IP to link to a “head” node and access all the other nodes through it.

In [ ]:
reserved_fip = chi.server.associate_floating_ip(server_id)
reserved_fip

### Security groups

The KVM cloud has a distinctive feature in the form of security groups, which are firewall rules that can be configured through OpenStack and the Horizon dashboard. They offer a hassle-free approach to configure the security of your VM. Although these groups also exist in the bare-metal cloud, they don’t serve any purpose there.

By default, all external connections to your VM are blocked. Therefore, to enable remote connections, you will need to assign the “Allow SSH” security group to your VM, which can be found by viewing the list of available groups.

It’s important to note that in almost all cases, the “Allow SSH” security group is the ONLY group that you need to assign to your VM.

The cell below make sure that there is an Allow SSH security group created, if there is no such groups it creates one.

In [ ]:
%%bash
export OS_AUTH_URL=https://kvm.tacc.chameleoncloud.org:5000/v3
export OS_REGION_NAME="KVM@TACC"

access_token=$(curl -s -H"authorization: token $JUPYTERHUB_API_TOKEN"     "$JUPYTERHUB_API_URL/users/$JUPYTERHUB_USER"     | jq -r .auth_state.access_token)
export OS_ACCESS_TOKEN="$access_token"
SECURITY_GROUP_NAME="Allow SSH"

if ! openstack security group show "$SECURITY_GROUP_NAME" > /dev/null 2>&1; then
    openstack security group create "$SECURITY_GROUP_NAME"  --description "Enable SSH traffic on TCP port 22"
    openstack security group rule create "$SECURITY_GROUP_NAME" \
     --protocol tcp --dst-port 22:22 --remote-ip 0.0.0.0/0


else
    echo "Security group already exists"
fi

In [ ]:
nova_server = chi.nova().servers.get(server_id)
nova_server.add_security_group("Allow SSH")
f"updated security groups: {[group.name for group in nova_server.list_security_group()]}"

Wait for the server to be ready to connect.

In [ ]:
server.wait_for_tcp(reserved_fip, port=22)

Now our resources are reserved and ready to login through SSH

## Exercise: log in to resources and execute commands

### Extracting the floating ip that is attached to our server

### Logging in over SSH via the jupyter env

In [ ]:
from chi.ssh import Remote

node = Remote(reserved_fip)
node.is_connected

**Executing terminal commands via notebook**

In [ ]:
node.run('echo "The connection is up"')
node.run('echo "Hello how are you" > hello.txt')

### Logging in over SSH via local terminal

In a local terminal on your own laptop, run

``` shell
user@username:~$ ssh cc@129.114.xxx.xxx
```

If your Chameleon key is not in the default location, you should also specify the path to your key as an argument, using -i.

``` shell
eg: ssh -i ~/.ssh/id_rsa cc@129.114.xx.xxx 
```

The output of the above command will look somewhat like this.

``` shell
Welcome to Ubuntu 20.04.4 LTS (GNU/Linux 5.4.0-124-generic x86_64)

 * Documentation:  https://help.ubuntu.com
 * Management:     https://landscape.canonical.com
 * Support:        https://ubuntu.com/advantage

  System information as of Thu Feb 23 17:52:13 UTC 2023

  System load:  0.08               Processes:             143
  Usage of /:   10.5% of 36.90GB   Users logged in:       1
  Memory usage: 12%                IPv4 address for ens3: 10.56.0.154
  Swap usage:   0%


0 updates can be applied immediately.


Last login: Thu Feb 23 16:44:05 2023 from 100.35.242.215
cc@cp3793-nyu-edu-fount:~$
```

Now we have been logged in to our remote host. we will run some commands to check the content of current directory

``` shell
:~$ ls
:~$ 
```

We can see that the root directory is empty.

We will create a directory named chameleon and then create a file hello.txt inside it.

``` shell
:~$ mkdir chameleon
:~$ cd chameleon
:~/chameleon$ 
:~/chameleon$ echo "hello Chameleon" > hello.txt
:~/chameleon$ 
```

We will use the file and directory created in the later exercises where we will see how transfering of file works between remote host and local host.

## Exercise: transfer files to and from resources

To securely transfer files between a local and a remote host, we will use the command-line tool called Secure Copy (SCP), which uses the Secure Shell (SSH) protocol for encryption and authentication. SCP provides a secure method for transferring files over a network, ensuring that data remains protected during the transfer process.

SCP is many times confusing, so to simplify it we can follow these rules:

1.  The path of the file or folder which is to be transfered should always come first.

2.  The path to which it has to be transfered comes after.

3.  Use -r when you are trying to transfer a directory of folder.

4.  use -i “key_path” when your key is not at the default location.

5.  When you are transfering a file from a remote host to your laptop, you will run *scp* from a terminal on your laptop.(Not on a terminal that is logged into the remote host)

### Using jupyter environment to transfer files vias *scp*

When we logged in via jupyter environment we created a file named hello.txt that is on our remote host, here we will run a *scp* command to get that file from remote to our jupyter environment.

In [ ]:
node.run("scp cc@{reserved_fip}:/home/cc/hello.txt .")


Now we have hello.txt in our jupyter environment we will make some changes to it by directly opening and changing it in the jupyter environment and then transfer the same file to the remote host.

In [ ]:
node.run("scp ~/work/hello.txt cc@{reserved_fip}:/home/cc/")


### Transfering files through the local terminal

When we logged in through our local environment on the terminal of our laptop, We created a folder “chameleon” and inside the folder we created a file “hello.txt” on the remote host. Here in this exercise we will run a *scp* command to get that file from remote host to our laptop.

``` shell
user@username:~$ scp cc@reserved_fip:/home/cc/chameleon/hello.txt .
hello.txt                       100%    1KB     0.1KB/s   00:00
user@username:~$
```

Run the code below and you will get the exact command which you have to use in your local terminal

In [ ]:
print(f'scp cc@{reserved_fip}:/home/cc/chameleon/hello.txt .')


Now we have transfered hello.txt from remote host to our laptop. Now we can open that file edit it in any of the editor and then try transfering the same to remote host.

``` shell
user@username:~$ scp hello.txt cc@reserved_fip:/home/cc/chameleon/
hello.txt                       100%    1KB     0.1KB/s   00:00
user@username:~$
```

Run the code below and you will get the exact command which you have to use in your local terminal to transfer the file back to remote host

In [ ]:
print(f'scp hello.txt cc@{reserved_fip}:/home/cc/chameleon/')


Use -i “key_path” if your Chameleon key is not in the default location.

Run the code below and you will get a similar command for transfering the file with key path when the file is not present at the default location.

In [ ]:
print(f'scp -i "~/.ssh/id_rsa_chameleon" hello.txt cc@{reserved_fip}:/home/cc/chameleon/')


## Exercise: delete resources

Once you are done using the resources, you can delete them by running the cell below. provide the input as “y” if you want to delete the resource.

In [ ]:
DELETE = False

if DELETE:
    chi.server.delete_server(server_id)
    ip_details = chi.network.get_floating_ip(reserved_fip)
    chi.neutron().delete_floatingip(ip_details["id"])